https://github.com/google/neuroglancer/issues/176#issuecomment-552032506

The current n5 support is rather limited. It only supports 3d volumes in the format described here:
https://github.com/saalfeldlab/n5-viewer

Your data source URL then needs to be of the form:

n5://http://localhost:5000/CONTAINER.n5/DATASET/c0

CONTAINER.n5/DATASET/attributes.json should contain something like:

{
"scales":[[1,1,1]],
"pixelResolution":{"unit":"um","dimensions":[0.097,0.097,0.18]}
}

CONTAINER.n5/DATASET/c0/attributes.json should not exist (or will be ignored if it does),

and

CONTAINER.n5/DATASET/c0/s0/attributes.json should be a normal n5 dataset attributes.json file (with raw or gzip compression).

The good news is that in a local branch I have already implemented better support for n5 that does not require such a specific directory structure, and I will be pushing it out soon. It is part of a large change that also adds support for 4-d, etc. volumes, but is not quite ready yet.

In [1]:
from fst.io import access

In [3]:
from skimage.morphology import ball
from skimage.transform import pyramid_gaussian

In [5]:
import napari
%gui qt

/home/bennettd/miniconda3/lib/python3.7/site-packages/napari/__init__.py:27: UserWarning: 
    napari was tested with QT library `>=5.12.3`.
    The version installed is 5.9.6. Please report any issues with this
    specific QT version at https://github.com/Napari/napari/issues.
    
  warn(message=warn_message)


In [101]:
base = ball(100).astype('float') 
base *= np.random.randint(0, 255, size=base.shape)
data = list(pyramid_gaussian(base))[:1]

In [102]:
data = [rescale(d, out_range=('uint8')).astype('uint8') for d in data]

In [103]:
scales = list([2 ** k] * 3 for k in range(len(data)))

In [111]:
attrs = dict(name='test', scales=scales, pixelResolution=dict(unit='um', dimensions=[1,1,1]))

In [112]:
from numcodecs import GZip
compressor = GZip(-1)

group = access('/data/tmp.n5/test/', mode='w')
group.attrs.update(**attrs)
subgroup = group.create_group(name='c0')
for ind, d in enumerate(data):
    arr = subgroup.array(name = f's{ind}', data=d, compressor=compressor)


In [106]:
from fst.io import read

In [108]:
read('/data/tmp.n5/test/c0/s0')[:].max()

255

In [109]:
napari.view_image(read('/data/tmp.n5/test/c0/s0'))

In [127]:
import dask.array as da
dat = da.from_array(read('/groups/cosem/cosem/data/HeLa_Cell2_4x4x4nm/HeLa_Cell2_4x4x4nm_raw.n5/volumes/raw/ch0'))

In [130]:
tmp = dat[:1000,:1000,:1000].compute()

In [131]:
napari.view_image(tmp)